# sentiment analysis 

Sentiment analysis is contextual mining of text which identifies and extracts subjective information in source material, and helping a business to understand the social sentiment of their brand, product or service while monitoring online conversations. 

Importing necessary packages

In [1]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

start the session

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = sparknlp.start()
spark.sparkContext.setLogLevel("ERROR")

22/03/21 15:46:29 WARN Utils: Your hostname, suvetha resolves to a loopback address: 127.0.1.1; using 192.168.0.231 instead (on interface wlo1)
22/03/21 15:46:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/suvetha/anaconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/suvetha/.ivy2/cache
The jars for the packages stored in: /home/suvetha/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-96d63249-d03d-487f-95b9-dafb03eff4da;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lo

preparing data into a format

In [3]:
documentAssembler = DocumentAssembler() \
     .setInputCol("text") \
     .setOutputCol("document") #assembling the data into the specified data

In [4]:
useEmbeddings = UniversalSentenceEncoder.pretrained() \
     .setInputCols(["document"]) \
     .setOutputCol("sentence_embeddings") #encodes text into high-dimensional vectors that can be used for text classification, semantic similarity, clustering and other natural language tasks

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ — ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]Download done! Loading the resource.
[ / ]

[ \ ]

2022-03-21 15:47:19.607734: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-21 15:47:20.003827: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294585000 Hz


[OK!]


In [5]:
sentiment = SentimentDLModel.pretrained("sentimentdl_use_twitter") \
    .setInputCols(["sentence_embeddings"]) \
    .setThreshold(0.7) \
    .setOutputCol("sentiment") #an annotator for multi-class sentiment analysis

sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ \ ]sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ]Download done! Loading the resource.
[OK!]


setting the pipeline

In [6]:
pipeline = Pipeline().setStages([
    documentAssembler,
    useEmbeddings,
    sentiment
])        #setting all the datas into a pipeline model

creating the dataframe

In [7]:
data = spark.createDataFrame([
    ["Wow, the new video is awesome!"],
    ["bruh what a damn waste of time"],
    ["It feels like empty"],
    ["Its ok to not to be okay"],
    ["am i happy or sad?"],
    ["hakunamatata"],
    ["i cant imagine what is going in my life"],
    ["good morning"],
    ["good and bad"]
]).toDF("text")   #dataset

fitting the pipeline

In [8]:
result = pipeline.fit(data).transform(data) #fitting to train and transform to change the data into uniform data
result.select("text", "sentiment.result").show(truncate=False)   #printing the sentiment result

+---------------------------------------+----------+
|text                                   |result    |
+---------------------------------------+----------+
|Wow, the new video is awesome!         |[positive]|
|bruh what a damn waste of time         |[negative]|
|It feels like empty                    |[negative]|
|Its ok to not to be okay               |[positive]|
|am i happy or sad?                     |[negative]|
|hakunamatata                           |[neutral] |
|i cant imagine what is going in my life|[negative]|
|good morning                           |[positive]|
|good and bad                           |[neutral] |
+---------------------------------------+----------+

